In [1]:
import re
import sys
from pyspark.rdd import RDD
import timeit, time
from pyspark import SparkContext, SparkConf
try:
    conf = SparkConf().setAppName("DT")
    sc = SparkContext(conf=conf) 
except:
    pass
sc   

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/18 14:30:13 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/05/18 14:30:13 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/05/18 14:30:13 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/05/18 14:30:13 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


<SparkContext master=yarn appName=DT>

In [2]:
#train_passenger_rdd= sc.textFile('train2.csv') 
#train_passenger_rdd.collect()
#print ("Train data size is {}".format(train_passenger_rdd.count()))

#test_passenger_rdd = sc.textFile('test2.csv') 
#test_passenger_rdd.collect()
#print ("Test data size is {}".format(test_passenger_rdd.count()))

from time import time


bucket_name="bdpp_bucket3"
path=f"gs://{bucket_name}/train2.csv"
t0 = time()
train_raw_data = sc.textFile(path)
tt = time() - t0
print ("Train data size is {}".format(train_raw_data.count()))
print("Time to read the training data {}".format(round(tt,3)))



path=f"gs://{bucket_name}/test2.csv"
t0 = time()
test_raw_data = sc.textFile(path)
tt= time() - t0
print ("Test data size is {}".format(test_raw_data.count()))
print("Time to read the test data {}".format(round(tt,3)))


22/05/18 14:30:25 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecutor.java:90)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1157)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)


Train data size is 103904
Time to read the training data 0.962


22/05/18 14:30:34 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecutor.java:90)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1157)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)


Test data size is 25976
Time to read the test data 0.084


In [3]:
from pyspark.mllib.regression import LabeledPoint
from numpy import array
#Create a map and split the data by ",".


data_passenger = train_raw_data.map(lambda x: x.split(';'))
data_test_passenger = test_raw_data.map(lambda x: x.split(';'))


data_passenger = data_passenger.map(lambda x: x[2:]) #deleting unecessary coloums
data_test_passenger =data_test_passenger.map(lambda x: x[2:]) #deleting unecessary coloums
#For normalization of some features
age = data_passenger.map(lambda x: x[2]).collect()
flight_distance = data_passenger.map(lambda x: x[5]).collect()
departure_delay = data_passenger.map(lambda x: x[20]).collect()
arrival_delay = data_passenger.map(lambda x: x[21]).collect() #already float

#Converting the normailization features to float
ageFloat= [float(x) for x in age]
flight_distanceFloat = [float(x) for x in flight_distance]
departure_delayFloat = [float(x) for x in departure_delay]
arrival_delayFloat  = [float(x) for x in arrival_delay]

#Finding min and max for the features
ageMax = max(ageFloat)
ageMin = min(ageFloat)

flight_distanceMax = max(flight_distanceFloat)
flight_distanceMin = min(flight_distanceFloat)

departure_delayMax = max(departure_delayFloat)
departure_delayMin = min(departure_delayFloat)

arrival_delayMax = max(arrival_delayFloat)
arrival_delayMin = min(arrival_delayFloat)

#Checking which coloums contains string
gender = data_passenger.map(lambda x: x[0]).distinct().collect()
customer_type = data_passenger.map(lambda x: x[1]).distinct().collect()
type_of_travel = data_passenger.map(lambda x: x[3]).distinct().collect()
flight_class = data_passenger.map(lambda x: x[4]).distinct().collect()




Now we have seperated the string values, we need to set them to integers. This is done by creating a label function to prepare the data.

In [4]:

def create_labeled_point(line_split):
    # leave_out = [21] as this is the label and rest are features
    clean_line_split = line_split[0:22]
    
    # convert protocol to numeric categorical variable
    try: 
        clean_line_split[0] = gender.index(clean_line_split[0])
    except:
        clean_line_split[0] = len(gender)
        
    # convert service to numeric categorical variable
    try:
        clean_line_split[1] = customer_type.index(clean_line_split[1])
    except:
        clean_line_split[1] = len(customer_type)
    
    # convert flag to numeric categorical variable
    try:
        clean_line_split[3] = type_of_travel.index(clean_line_split[3])
    except:
        clean_line_split[3] = len(type_of_travel)
        
         # convert flag to numeric categorical variable
    try:
        clean_line_split[4] = flight_class.index(clean_line_split[4])
    except:
        clean_line_split[4] = len(flight_class)
        
        
    clean_line_split[2] =(float(clean_line_split[2]) - ageMin)/(ageMax-ageMin)
    clean_line_split[5] =(float(clean_line_split[5]) - flight_distanceMin)/(flight_distanceMax-flight_distanceMin)
    clean_line_split[20] =(float(clean_line_split[20]) - departure_delayMin)/(departure_delayMax-departure_delayMin)
    clean_line_split[21] =(float(clean_line_split[21]) - arrival_delayMin)/(arrival_delayMax-arrival_delayMin)

    
    # convert label to binary label
    reviews = 1.0
    if line_split[22]=='neutral or dissatisfied':
        reviews = 0.0
        
        
    return LabeledPoint(reviews, array([float(x) for x in clean_line_split]))



training_passenger = data_passenger.map(create_labeled_point).repartition(12)
testing_passenger = data_test_passenger.map(create_labeled_point).repartition(12)
print('Number of partitions: {}'.format(training_passenger.getNumPartitions()))

#class1_test = training_passenger.map(lambda x: (x.label, 1)).reduceByKey(lambda x, y: x+y)
#class1_test.collect()


Number of partitions: 12


We create a decicion tree classifier which we can then use to predict our test data of the passenger reviews

Classifying using the decision tree algorithm

In [5]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from time import time

# Build the model
t0 = time()
tree_model = DecisionTree.trainClassifier(training_passenger, numClasses=2, 
                                         categoricalFeaturesInfo={0: len(gender), 1: len(customer_type), 3: len(type_of_travel), 4: len(flight_class)},
                                          impurity='gini', maxDepth=4, maxBins=100)
tt = time() - t0

print ("Decision tree classifier trained in {} seconds".format(round(tt,3)))



Decision tree classifier trained in 13.175 seconds


Now we predict our test dataset and time it!

In [6]:
predictions = tree_model.predict(testing_passenger.map(lambda p: p.features))
labels_and_preds = testing_passenger.map(lambda p: p.label).zip(predictions)


In [7]:
t0 = time()
test_accuracy = labels_and_preds.filter(lambda v: v[0] == v[1]).count() / float(testing_passenger.count())
tt = time() - t0

print ("Prediction for decision tree made in {} seconds. Test accuracy is {}".format(round(tt,3), round(test_accuracy,4)))

Prediction for decision tree made in 11.11 seconds. Test accuracy is 0.888


In [8]:
print ("Learned classification tree model:")
print (tree_model.toDebugString())

Learned classification tree model:
DecisionTreeModel classifier of depth 4 with 19 nodes
  If (feature 11 <= 3.5)
   If (feature 6 <= 0.5)
    If (feature 13 <= 0.5)
     Predict: 0.0
    Else (feature 13 > 0.5)
     Predict: 1.0
   Else (feature 6 > 0.5)
    If (feature 6 <= 3.5)
     Predict: 0.0
    Else (feature 6 > 3.5)
     If (feature 6 <= 4.5)
      Predict: 0.0
     Else (feature 6 > 4.5)
      Predict: 1.0
  Else (feature 11 > 3.5)
   If (feature 3 in {0.0})
    If (feature 6 <= 4.5)
     Predict: 0.0
    Else (feature 6 > 4.5)
     Predict: 1.0
   Else (feature 3 not in {0.0})
    If (feature 13 <= 3.5)
     If (feature 11 <= 4.5)
      Predict: 0.0
     Else (feature 11 > 4.5)
      Predict: 1.0
    Else (feature 13 > 3.5)
     Predict: 1.0



Classifying using the logistic regression algorithm

In [9]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint

t0 = time()
# Build the model
model = LogisticRegressionWithLBFGS.train(training_passenger)

tt = time() - t0
print ("Logistic regression classifier trained in {} seconds".format(round(tt,3)))
model.weights


22/05/18 14:31:24 WARN org.apache.spark.ml.util.Instrumentation: [bcbfd60d] Initial coefficients will be ignored! Its dimensions (1, 22) did not match the expected size (1, 22)
22/05/18 14:31:26 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/05/18 14:31:26 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


Logistic regression classifier trained in 33.577 seconds


DenseVector([-0.4564, -2.1474, -2.8569, 1.6176, 0.6422, -0.7048, 0.4154, -0.2221, -0.2139, -0.2523, -0.3615, 0.4153, -0.0907, 0.5352, 0.1304, 0.0327, -0.1647, 0.0929, -0.2449, -0.0388, 5.6949, -15.9805])

In [10]:
predictions = model.predict(testing_passenger.map(lambda p: p.features))
labels_and_preds = testing_passenger.map(lambda p: p.label).zip(predictions)


t0 = time()
test_accuracy = labels_and_preds.filter(lambda v: v[0] == v[1]).count() / float(testing_passenger.count())
tt = time() - t0

print ("Prediction for logistic regression made in {} seconds. Test accuracy is {}".format(round(tt,3), round(test_accuracy,4)))


Prediction for logistic regression made in 1.18 seconds. Test accuracy is 0.8213


Random forrest classifier

In [11]:
from pyspark.mllib.tree import RandomForest, RandomForestModel

t0 = time()
R_model = RandomForest.trainClassifier(training_passenger, numClasses=2, categoricalFeaturesInfo={0: len(gender), 1: len(customer_type), 3: len(type_of_travel), 4: len(flight_class)},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=100)
tt = time() - t0

print ("Random forrest classifier trained in {} seconds".format(round(tt,3)))



Random forrest classifier trained in 6.661 seconds


In [12]:
predictions = R_model.predict(testing_passenger.map(lambda x: x.features))
labelsAndPredictions = testing_passenger.map(lambda lp: lp.label).zip(predictions)


t0 = time()
test_accuracy = labelsAndPredictions.filter(lambda v: v[0] == v[1]).count() / float(testing_passenger.count())
tt = time() - t0

print ("Prediction for Random forrest made in {} seconds. Test accuracy is {}".format(round(tt,3), round(test_accuracy,4)))


Prediction for Random forrest made in 1.806 seconds. Test accuracy is 0.8588


Naive bayes (BONUS)

In [13]:
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel

t0 = time()

# Train a naive Bayes model.
Naive_model = NaiveBayes.train(training_passenger, 1.0)
tt = time() - t0


print ("Naive bayes classifier trained in {} seconds".format(round(tt,3)))


Naive bayes classifier trained in 5.223 seconds


In [14]:
predictionAndLabel = testing_passenger.map(lambda p: (Naive_model.predict(p.features), p.label))

t0 = time()

accuracy = 1.0 * predictionAndLabel.filter(lambda pl: pl[0] == pl[1]).count() / testing_passenger.count()
tt = time() - t0


print ("Prediction for Naive Bayes made in {} seconds. Test accuracy is {}".format(round(tt,3), round(accuracy,4)))


Prediction for Naive Bayes made in 0.809 seconds. Test accuracy is 0.7833


SVM algorithm (BONUS)

In [15]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel

t0 = time()

# Build the model
svm = SVMWithSGD.train(training_passenger)

tt = time() - t0
print ("SVM classifier trained in {} seconds".format(round(tt,3)))

SVM classifier trained in 15.915 seconds


In [16]:
predictions = svm.predict(testing_passenger.map(lambda p: p.features))
labels_and_preds = testing_passenger.map(lambda p: p.label).zip(predictions)


t0 = time()
test_accuracy = labels_and_preds.filter(lambda v: v[0] == v[1]).count() / float(testing_passenger.count())
tt = time() - t0

print ("Prediction for SVM made in {} seconds. Test accuracy is {}".format(round(tt,3), round(test_accuracy,4)))



Prediction for SVM made in 0.978 seconds. Test accuracy is 0.6694
